In [ ]:
import os, random
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from tqdm import trange
import ale_py

class DQNBreakout(gym.Wrapper):
    def __init__(self, render_mode='human', repeat=4, device='cpu'):
        env = gym.make("ALE/Breakout-v5", render_mode=render_mode)
        super().__init__(env)
        self.repeat = repeat
        self.device = device
        self.frame_buffer = []

    def reset(self):
        obs, info = self.env.reset()
        self.frame_buffer = [obs]
        tensor_state = (
            torch.from_numpy(obs)
                 .permute(2, 0, 1)
                 .unsqueeze(0)
                 .to(self.device)
                 .float()
                 .div(255.0)
        )
        return tensor_state, info

    def step(self, action):
        total_reward = 0
        terminated = False
        truncated = False
        for _ in range(self.repeat):
            obs, reward, terminated, truncated, info = self.env.step(action)
            total_reward += reward
            self.frame_buffer.append(obs)
            if terminated or truncated:
                break
        max_frame = np.max(self.frame_buffer[-2:], axis=0) # Max-pooling över frames
        tensor_frame = (
            torch.from_numpy(max_frame)
                 .permute(2, 0, 1)
                 .unsqueeze(0)
                 .to(self.device)
                 .float()
                 .div(255.0)
        )
        return tensor_frame, total_reward, terminated or truncated, info

    def render(self):
        return self.env.render()

class SimpleDQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super().__init__()
        c, h, w = input_shape
        self.conv = nn.Sequential(
            nn.Conv2d(c, 16, kernel_size=8, stride=4), 
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=4, stride=2), 
            nn.ReLU(),
        )
        def conv_out(size, kernel, stride):
            return (size - (kernel-1) -1) // stride + 1
        h1 = conv_out(h, 8, 4); 
        h2 = conv_out(h1, 4, 2)
        w1 = conv_out(w, 8, 4); 
        w2 = conv_out(w1, 4, 2)
        lin_input = h2 * w2 * 32
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(lin_input, 256), nn.ReLU(),
            nn.Linear(256, n_actions) # Ger Q(framtida belöning) värde för varje möjlig action [Q(s,a1), Q(s,a2)...]
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

class ReplayBuffer:
    def __init__(self, capacity):
        self.buf = deque(maxlen=capacity)
    def push(self, s,a,r,s2,d): #(nuvarande state, action, rward, nästa state givet a, done)
        self.buf.append((s,a,r,s2,d))
    def sample(self, batch_size):
        batch = random.sample(self.buf, batch_size) # Random batch -> stabil lärning / undvika lokala minimum ?
        states,actions, rewards, next_states, dones = zip(*batch)
        return (torch.cat(states), 
                torch.tensor(actions, device=device),
                torch.tensor(rewards, device=device, dtype=torch.float),
                torch.cat(next_states),
                torch.tensor(dones, device=device, dtype=torch.uint8))
    def __len__(self):
        return len(self.buf)

def select_action(net, state, eps, n_actions): # epsilon greedy
    if random.random() < eps:
        return random.randrange(n_actions) # här väljs en random action <- exploration
    with torch.no_grad():
        q = net(state)
        return q.argmax(dim=1).item() # exploitation


# Träning 

if __name__ == "__main__":
    os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    env = DQNBreakout(render_mode='human', device=device)
    state, _ = env.reset()
    _, c, h, w = state.shape
    n_actions = env.action_space.n

    policy_net = SimpleDQN((c, h, w), n_actions).to(device)
    target_net = SimpleDQN((c, h, w), n_actions).to(device)
    target_net.load_state_dict(policy_net.state_dict())

    optimizer = optim.Adam(policy_net.parameters(), lr=1e-4)
    buffer = ReplayBuffer(capacity=100_000)

    # Kortare träning: 5 000 steg
    num_steps = 58_000
    batch_size = 32
    gamma = 0.99
    target_update = 1_000

    episode_reward = 0
    best_reward = -float('inf')
    for step in trange(num_steps, desc="Training DQN"):
        # e-greedy
        eps = max(0.01, 1 - step / (num_steps * 0.5))
        action = select_action(policy_net, state, eps, n_actions)

        # step
        next_state, reward, done, _ = env.step(action)
        episode_reward += reward
        buffer.push(state, action, reward, next_state, done)
        state = next_state

        # c) träning
        if len(buffer) >= batch_size:
            states, actions, rewards, next_states, dones = buffer.sample(batch_size)
            q_vals = policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
            with torch.no_grad():
                next_q = target_net(next_states).max(1)[0]
                target = rewards + gamma * next_q * (1 - dones) #Bellman
            loss = nn.MSELoss()(q_vals, target)
            optimizer.zero_grad();
            loss.backward(); 
            optimizer.step()

        
        if step % target_update == 0:
            target_net.load_state_dict(policy_net.state_dict())

        if done:           
            print(f"\nEpisode slut – belöning: {episode_reward:.1f}")
            if episode_reward > best_reward:
                best_reward = episode_reward
                torch.save(policy_net.state_dict(), "best_model.pth")
            state, _ = env.reset()
            episode_reward = 0

    print("Träning klar!")


Training DQN:   0%|          | 39/58000 [00:19<8:51:06,  1.82it/s] 


Episode slut – belöning: 0.0


Training DQN:   0%|          | 89/58000 [00:53<9:25:46,  1.71it/s] 


Episode slut – belöning: 1.0


Training DQN:   0%|          | 142/58000 [01:27<8:12:08,  1.96it/s] 

...

Training DQN:  54%|█████▍    | 31341/58000 [2:53:32<2:05:10,  3.55it/s]


Episode slut – belöning: 6.0


Training DQN:  54%|█████▍    | 31454/58000 [2:54:11<2:02:51,  3.60it/s]


Episode slut – belöning: 10.0


Training DQN:  54%|█████▍    | 31483/58000 [2:54:21<2:37:56,  2.80it/s]